# Setup

In [ ]:
!nvidia-smi

In [ ]:
import os
os.cpu_count()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/

In [ ]:
# !rm -r netFound

In [ ]:
# !git clone https://github.com/SNL-UCSB/netFound.git

In [ ]:
cd netFound

In [ ]:
!apt-get update && apt-get upgrade -y
!apt-get install -y --no-install-recommends gcc python3-dev wget graphviz parallel nano make cmake g++ libpcap-dev

In [ ]:
!pip install --no-cache-dir -r requirements.txt

In [ ]:
!rm -rf '/usr/local/pcapplusplus-24.09-ubuntu-22.04-gcc-11.4.0-x86_64'

In [ ]:
!wget https://github.com/seladb/PcapPlusPlus/releases/download/v24.09/pcapplusplus-24.09-ubuntu-22.04-gcc-11.4.0-x86_64.tar.gz
!tar -xvf pcapplusplus-24.09-ubuntu-22.04-gcc-11.4.0-x86_64.tar.gz
!rm pcapplusplus-24.09-ubuntu-22.04-gcc-11.4.0-x86_64.tar.gz
!mv pcapplusplus-24.09-ubuntu-22.04-gcc-11.4.0-x86_64 /usr/local/
!ln -s /usr/local/pcapplusplus-24.09-ubuntu-22.04-gcc-11.4.0-x86_64 /usr/local/pcapplusplus

In [ ]:
import os
os.environ["PATH"] += os.pathsep + "/usr/local/pcapplusplus/bin"

In [ ]:
!find . -type f -name "*.sh"

In [ ]:
!chmod +x ./scripts/finetune-test.sh
!chmod +x ./scripts/pretrain-test.sh
!chmod +x ./src/pre_process/1_filter.sh
!chmod +x ./src/pre_process/2_pcap_splitting.sh
!chmod +x ./src/pre_process/3_extract_fields.sh

In [ ]:
PREPROCESS_SRC_DIR = "src/pre_process/packets_processing_src"
PREPROCESS_BUILD_DIR = "build/preprocess"
DATA_DIR = "data/test"

In [ ]:
!ls src/pre_process/

In [ ]:
!rm -rf {PREPROCESS_BUILD_DIR}
!rm -f src/pre_process/1_filter
!rm -f src/pre_process/3_field_extraction

In [ ]:
!mkdir -p {PREPROCESS_BUILD_DIR}

In [ ]:
!cmake -S {PREPROCESS_SRC_DIR} -B {PREPROCESS_BUILD_DIR}

In [ ]:
!make -C {PREPROCESS_BUILD_DIR}

In [ ]:
!cp {PREPROCESS_BUILD_DIR}/1_filter src/pre_process/1_filter

In [ ]:
!cp {PREPROCESS_BUILD_DIR}/3_field_extraction src/pre_process/3_field_extraction

In [ ]:
!ls {DATA_DIR}/finetuning/raw/*

In [ ]:
!chmod +x src/pre_process/1_filter
!chmod +x src/pre_process/3_field_extraction

# Pretraining

In [ ]:
!nvidia-smi

In [ ]:
data_folder = "data/test/finetuning/final/combined/"
model_output_dir = "models/test/pretraining/pretrained_model/"

In [ ]:
!mkdir -p {model_output_dir}

In [ ]:
!rm -rf {model_output_dir}/*

In [ ]:
!python src/train/NetfoundPretraining.py --train_dir {data_folder} --output_dir {model_output_dir} --report_to tensorboard --do_train --num_train_epochs 50 --overwrite_output_dir --save_safetensors false --mlm_probability 0.10 --learning_rate 2e-5 --do_eval --validation_split_percentage 30

In [ ]:
!ls {model_output_dir}

In [ ]:
# from google.colab import runtime
# runtime.unassign()

# Finetuning

In [ ]:
!nvidia-smi

In [ ]:
data_folder = "data/test/finetuning/final/combined/"
pretrained_model_dir = "models/test/pretraining/pretrained_model"
model_output_dir = "models/test/finetuning/finetuned_model"

In [ ]:
!mkdir -p {model_output_dir}

In [ ]:
!rm -rf {model_output_dir}/*

In [ ]:
!python src/train/NetfoundFinetuning.py --train_dir {data_folder} --test_dir {data_folder} --model_name_or_path {pretrained_model_dir} --output_dir {model_output_dir} --report_to tensorboard --overwrite_output_dir --save_safetensors false --do_train --do_eval --eval_strategy epoch --save_strategy epoch --learning_rate 0.01 --num_train_epochs 10 --problem_type single_label_classification --num_labels 6 --load_best_model_at_end

In [ ]:
!ls {model_output_dir}

In [ ]:
# from google.colab import runtime
# runtime.unassign()